In [1]:
import numpy as np
import sys
import os

In [1]:
list1 =[] #list of barriers to be compared against dft
KIMID = "EAM_Dynamo_Mishin_2004_NiAl__MO_101214310689_005"
potentialAbbreviation = "Mishin04"

In [2]:
# These 2 write files to relax cells to the lattice parameters of the given potential
def stateRelax(outfileName,KIMID,cutDownCell,lammpsDataFileName):
    #Once run the output of this file should be used as the input for nebFile
    #outfile = "CutdownBarriers_Mishin04{}_relax_final.lmp".format((j+1))
    f = open(outfileName, "w")
    input_text = "kim init "+KIMID+ "metal \n \
            atom_style atomic \n \
            atom_modify map array sort 0 0.0 \n \
            read_data "+cutDownCell+ "\n \
            mass 1 26.981539 \n \
            mass 2 58.6934 \n \
            kim interactions Al Ni \n \
            neighbor 0.3 bin \n \
            neigh_modify delay 0 every 1 check yes \n \
            fix 1 all box/relax iso 0.0 vmax 0.001 \n \
            minimize 1.0e-7 1.0e-5 1000 10000 \n \
            write_data relaxed_"+lammpsDataFileName
    
    
    f.write(input_text)
    

    f.close()

for j in list1:
    stateRelax(potentialAbbreviation+"step{}_relax_initial.lmp".format(j),KIMID,"cutDownCellStep{}initial.txt".format(j),potentialAbbreviation+"_step{}initial.txt".format(j))
    stateRelax(potentialAbbreviation+"step{}_relax_final.lmp".format(j),KIMID,"cutDownCellStep{}final.txt".format(j),potentialAbbreviation+"_step{}final.txt".format(j))

In [ ]:
#This cell requires the output from the above cells to be ran in lammps
def nebFile(fileName,outputFileName):    
    
    f = open(outputFileName, "w")
    f.write("255 \n")

    g = open(fileName , "r")
    glines = g.readlines()
    for i in range(len(glines)-11):
        gline = glines[11+i].split()
        del gline[1]
        fline = gline
        #print(fline)
        for item in fline:
            f.write(" %s" % item)
        f.write("\n")

    f.close()

for j in list1:
    nebFile("relaxed_"+potentialAbbreviation+"_step{}final.txt".format(j),"NEBcutstep"+potentialAbbreviation+"{}.txt".format(j)) # This is how it should be used in current context

In [ ]:
# This writes the files for the neb calculation  

def nebExcuteable(outfileName,KIMID,initialFile,finalFile):   
    f = open(outfileName, "w")
    input_text = "kim init" + KIMID + "metal \n \
            atom_style atomic \n \
            atom_modify map array sort 0 0.0 \n \
            read_data "+initialFile+" \n \
            mass 1 26.981539 \n \
            mass 2 58.6934 \n \
            kim interactions Al Ni \n \
            neighbor 0.3 bin \n \
            neigh_modify delay 0 every 1 check yes \n \
            minimize 1.0e-7 1.0e-5 1000 10000 \n \
            fix 1 all neb 1.0 parallel ideal #end last/efirst 1 \n \
            min_style  quickmin \n \
        neb 1.0e-7 1.0e-5 10000 10000 50 final "+finalFile # The file in this line is written by nebFile
    #print(input_text)
    
    f.write(input_text)


    f.close()

    for j in list1:
        nebExcuteable(potentialAbbreviation+"step{}.lmp".format(j),KIMID,potentialAbbreviation+"{}_relax_initial.lmp".format(j),"NEBcutstep"+potentialAbbreviation+"{}.txt".format(j))